In [8]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Richa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [26]:
news_dataset = pd.read_csv('FakeNewsNet.csv')
news_dataset.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [27]:
news_dataset.shape

(23196, 5)

In [28]:
news_dataset.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [29]:
news_dataset = news_dataset.fillna('')
print(news_dataset)

                                                   title  \
0      Kandi Burruss Explodes Over Rape Accusation on...   
1      People's Choice Awards 2018: The best red carp...   
2      Sophia Bush Sends Sweet Birthday Message to 'O...   
3      Colombian singer Maluma sparks rumours of inap...   
4      Gossip Girl 10 Years Later: How Upper East Sid...   
...                                                  ...   
23191  Pippa Middleton wedding: In case you missed it...   
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...   
23193  Jessica Chastain Recalls the Moment Her Mother...   
23194  Tristan Thompson Feels "Dumped" After Khloé Ka...   
23195  Kelly Clarkson Performs a Medley of Kendrick L...   

                                                news_url  \
0      http://toofab.com/2017/05/08/real-housewives-a...   
1      https://www.today.com/style/see-people-s-choic...   
2      https://www.etonline.com/news/220806_sophia_bu...   
3      https://www.dailymail.co.uk/news

In [30]:
news_dataset['content'] = news_dataset['source_domain']+ ' ' +news_dataset['title']
print(news_dataset['content'])

0        toofab.com Kandi Burruss Explodes Over Rape Ac...
1        www.today.com People's Choice Awards 2018: The...
2        www.etonline.com Sophia Bush Sends Sweet Birth...
3        www.dailymail.co.uk Colombian singer Maluma sp...
4        www.zerchoo.com Gossip Girl 10 Years Later: Ho...
                               ...                        
23191    www.express.co.uk Pippa Middleton wedding: In ...
23192    hollywoodlife.com Zayn Malik & Gigi Hadid’s Sh...
23193    www.justjared.com Jessica Chastain Recalls the...
23194    www.intouchweekly.com Tristan Thompson Feels "...
23195    www.billboard.com Kelly Clarkson Performs a Me...
Name: content, Length: 23196, dtype: object


In [34]:
X = news_dataset.drop(columns='real', axis=1)
Y = news_dataset['real']
print(X)
print(Y)

                                                   title  \
0      Kandi Burruss Explodes Over Rape Accusation on...   
1      People's Choice Awards 2018: The best red carp...   
2      Sophia Bush Sends Sweet Birthday Message to 'O...   
3      Colombian singer Maluma sparks rumours of inap...   
4      Gossip Girl 10 Years Later: How Upper East Sid...   
...                                                  ...   
23191  Pippa Middleton wedding: In case you missed it...   
23192  Zayn Malik & Gigi Hadid’s Shocking Split: Why ...   
23193  Jessica Chastain Recalls the Moment Her Mother...   
23194  Tristan Thompson Feels "Dumped" After Khloé Ka...   
23195  Kelly Clarkson Performs a Medley of Kendrick L...   

                                                news_url  \
0      http://toofab.com/2017/05/08/real-housewives-a...   
1      https://www.today.com/style/see-people-s-choic...   
2      https://www.etonline.com/news/220806_sophia_bu...   
3      https://www.dailymail.co.uk/news

In [40]:
port_stem = PorterStemmer()


In [38]:
# streaming: it is the process reducing the words to its root word
# eg:- actor, actress, acting ----------in these words act is a root word

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [41]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [58]:
print(news_dataset['content'])

0        toofab com kandi burruss explod rape accu real...
1        www today com peopl choic award best red carpe...
2        www etonlin com sophia bush send sweet birthda...
3        www dailymail co uk colombian singer maluma sp...
4        www zerchoo com gossip girl year later upper e...
                               ...                        
23191    www express co uk pippa middleton wed case mis...
23192    hollywoodlif com zayn malik gigi hadid shock s...
23193    www justjar com jessica chastain recal moment ...
23194    www intouchweekli com tristan thompson feel du...
23195    www billboard com kelli clarkson perform medle...
Name: content, Length: 23196, dtype: object


In [59]:
X = news_dataset['content'].values
Y = news_dataset['real'].values
print(X)
print(Y)

['toofab com kandi burruss explod rape accu real housew atlanta reunion video'
 'www today com peopl choic award best red carpet look'
 'www etonlin com sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva'
 ...
 'www justjar com jessica chastain recal moment mother boyfriend slap kick genit'
 'www intouchweekli com tristan thompson feel dump khlo kardashian refu let move la home exclu'
 'www billboard com kelli clarkson perform medley kendrick lamar humbl hit billboard music award']
[1 1 1 ... 1 0 1]


In [62]:
Y.shape

(23196,)

In [64]:
# converting textual data into numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
print(X)

  (0, 13591)	0.21108934405813048
  (0, 12894)	0.29467349125048065
  (0, 10494)	0.2543303205934995
  (0, 10203)	0.23128301802698972
  (0, 10151)	0.30215839846863407
  (0, 6597)	0.3874939993343693
  (0, 5845)	0.2612462026749161
  (0, 4159)	0.3550616497873391
  (0, 2447)	0.05082186658865081
  (0, 1698)	0.3874939993343693
  (0, 645)	0.3073756565758506
  (0, 56)	0.2597097788538541
  (1, 14182)	0.10189043319535993
  (1, 12860)	0.36817044163275126
  (1, 10274)	0.37702288594702094
  (1, 9409)	0.23697414575567807
  (1, 7380)	0.33284454859410584
  (1, 2447)	0.08159478582438673
  (1, 2208)	0.42251489448694257
  (1, 1906)	0.3875096512688879
  (1, 1093)	0.33844294892026705
  (1, 722)	0.3083771474680274
  (2, 14182)	0.05442978142050365
  (2, 13031)	0.30452204896780216
  (2, 12300)	0.2331559845985754
  :	:
  (23194, 10318)	0.31860580642240854
  (23194, 8327)	0.28221547915154177
  (23194, 7179)	0.3087048153381826
  (23194, 6943)	0.26381957644897924
  (23194, 6746)	0.2878505190105094
  (23194, 6610)	0.

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)
model = LogisticRegression()
model.fit(X_train, Y_train)


LogisticRegression()

In [69]:
X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_train)
print("accuracy score of the data :",training_data_accuracy )

accuracy score of the data : 0.8828411295537831


In [71]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==0):
    print('the news is real')
else:
    print('the news is fake')

[1]
the news is fake
